In [12]:
import pyspark
from pyspark.sql import SparkSession

In [26]:
spark = SparkSession.builder \
    .master("local[*]") \
    .appName('test') \
    .getOrCreate()

In [27]:
df = spark.read.option("header", "true").parquet("../data/yellow_tripdata_2022-01.parquet")

In [28]:
df.show()

+--------+--------------------+---------------------+---------------+-------------+----------+------------------+------------+------------+------------+-----------+-----+-------+----------+------------+---------------------+------------+--------------------+-----------+
|VendorID|tpep_pickup_datetime|tpep_dropoff_datetime|passenger_count|trip_distance|RatecodeID|store_and_fwd_flag|PULocationID|DOLocationID|payment_type|fare_amount|extra|mta_tax|tip_amount|tolls_amount|improvement_surcharge|total_amount|congestion_surcharge|airport_fee|
+--------+--------------------+---------------------+---------------+-------------+----------+------------------+------------+------------+------------+-----------+-----+-------+----------+------------+---------------------+------------+--------------------+-----------+
|       1| 2022-01-01 07:35:40|  2022-01-01 07:53:29|            2.0|          3.8|       1.0|                 N|         142|         236|           1|       14.5|  3.0|    0.5|      3.6

In [29]:
#Check shape of the dataframe
print((df.count(), len(df.columns)))

(2463931, 19)


In [30]:
!wc -l ../data/yellow_tripdata_2022-01.parquet

122909 ../data/yellow_tripdata_2022-01.parquet


In [31]:
df.printSchema()

root
 |-- VendorID: long (nullable = true)
 |-- tpep_pickup_datetime: timestamp (nullable = true)
 |-- tpep_dropoff_datetime: timestamp (nullable = true)
 |-- passenger_count: double (nullable = true)
 |-- trip_distance: double (nullable = true)
 |-- RatecodeID: double (nullable = true)
 |-- store_and_fwd_flag: string (nullable = true)
 |-- PULocationID: long (nullable = true)
 |-- DOLocationID: long (nullable = true)
 |-- payment_type: long (nullable = true)
 |-- fare_amount: double (nullable = true)
 |-- extra: double (nullable = true)
 |-- mta_tax: double (nullable = true)
 |-- tip_amount: double (nullable = true)
 |-- tolls_amount: double (nullable = true)
 |-- improvement_surcharge: double (nullable = true)
 |-- total_amount: double (nullable = true)
 |-- congestion_surcharge: double (nullable = true)
 |-- airport_fee: double (nullable = true)



In [32]:
type(df)

pyspark.sql.dataframe.DataFrame

In [33]:
df.head(5)

[Row(VendorID=1, tpep_pickup_datetime=datetime.datetime(2022, 1, 1, 7, 35, 40), tpep_dropoff_datetime=datetime.datetime(2022, 1, 1, 7, 53, 29), passenger_count=2.0, trip_distance=3.8, RatecodeID=1.0, store_and_fwd_flag='N', PULocationID=142, DOLocationID=236, payment_type=1, fare_amount=14.5, extra=3.0, mta_tax=0.5, tip_amount=3.65, tolls_amount=0.0, improvement_surcharge=0.3, total_amount=21.95, congestion_surcharge=2.5, airport_fee=0.0),
 Row(VendorID=1, tpep_pickup_datetime=datetime.datetime(2022, 1, 1, 7, 33, 43), tpep_dropoff_datetime=datetime.datetime(2022, 1, 1, 7, 42, 7), passenger_count=1.0, trip_distance=2.1, RatecodeID=1.0, store_and_fwd_flag='N', PULocationID=236, DOLocationID=42, payment_type=1, fare_amount=8.0, extra=0.5, mta_tax=0.5, tip_amount=4.0, tolls_amount=0.0, improvement_surcharge=0.3, total_amount=13.3, congestion_surcharge=0.0, airport_fee=0.0),
 Row(VendorID=2, tpep_pickup_datetime=datetime.datetime(2022, 1, 1, 7, 53, 21), tpep_dropoff_datetime=datetime.dateti

In [34]:
#Repartition the dataframe to 10 partitions
df = df.repartition(10)


In [36]:
#Check the number of partitions
df.rdd.getNumPartitions()

10

In [37]:
#Check the size of each partition
df.rdd.glom().map(len).collect()



[246393,
 246393,
 246393,
 246393,
 246394,
 246393,
 246393,
 246393,
 246393,
 246393]

In [38]:
#Save the partitioned dataframe as parquet file
df.write.parquet("partition_yellow/2022/01/")

23/05/23 21:10:20 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 95.00% for 8 writers
23/05/23 21:10:20 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 84.44% for 9 writers
23/05/23 21:10:20 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 76.00% for 10 writers


23/05/23 21:10:23 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 84.44% for 9 writers
23/05/23 21:10:23 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 95.00% for 8 writers
